In [ ]:
pip install pymc3

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from abc import ABC, abstractmethod

import matplotlib.pyplot as plt
import seaborn as sns


import sklearn
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
accounts = pd.read_csv('./accounts.csv',encoding='utf-8')
orders = pd.read_csv('./orders.csv',encoding='utf-8')
products = pd.read_csv('./products.csv',encoding='utf-8', dtype={'published_at':str})

In [3]:
clicks = pd.read_csv('./click.csv',encoding='utf-8')

In [4]:
accounts.isnull().sum()

account_id        0
gender            0
age               0
address           0
last_login_dts    0
dtype: int64

In [5]:
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670458 entries, 0 to 1670457
Data columns (total 5 columns):
account_id        1670458 non-null int64
gender            1670458 non-null object
age               1670458 non-null float64
address           1670458 non-null object
last_login_dts    1670458 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 63.7+ MB


In [6]:
clicks.isnull().sum()

request_date_time    0
account_id           0
device_type          0
product_id           0
before_product_id    0
dtype: int64

In [7]:
clicks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60265430 entries, 0 to 60265429
Data columns (total 5 columns):
request_date_time    object
account_id           int64
device_type          object
product_id           int64
before_product_id    int64
dtypes: int64(3), object(2)
memory usage: 2.2+ GB


In [8]:
orders.isnull().sum()

order_id      0
account_id    0
product_id    0
price         0
created_at    0
dtype: int64

In [9]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8382514 entries, 0 to 8382513
Data columns (total 5 columns):
order_id      int64
account_id    int64
product_id    int64
price         float64
created_at    int64
dtypes: float64(1), int64(4)
memory usage: 319.8 MB


In [10]:
products.isnull().sum()

product_id      0
product_name    0
category_id     0
published_at    0
shop_price      0
maker_name      0
dtype: int64

In [11]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725489 entries, 0 to 1725488
Data columns (total 6 columns):
product_id      int64
product_name    object
category_id     int64
published_at    object
shop_price      float64
maker_name      object
dtypes: float64(1), int64(2), object(3)
memory usage: 79.0+ MB


In [12]:
accounts.head()

,account_id,gender,age,address,last_login_dts
0,102600,F,50.0,경상북도 포항시 남구 지곡동 효자그린1차아파트,1.591700e+12
1,103417,F,40.0,경기 부천시 원미 심곡동,1.589295e+12
2,105247,M,59.0,광주광역시 북구 일곡동 816-3 동아아파트,1.579497e+12
3,109061,F,59.0,강원 동해시 천곡동 센트로빌아파트,1.584095e+12
4,109217,F,46.0,경기 수원시 영통구 영통1동 황골마을1단지아파트,1.583754e+12


In [13]:
index_1 = accounts[accounts.gender=='-'].index
index_2 = accounts[accounts.gender=='0'].index

accounts = accounts.drop(index_1)
accounts = accounts.drop(index_2)

In [14]:
accounts.age = accounts.age.astype(int)
under_7 = list(accounts[accounts.age <= 7].index)
accounts = accounts.drop(under_7)

In [15]:
accounts.address.str.split().str[0]
accounts['capital_area'] = accounts.address.str.split().str[0]

In [16]:
accounts.iloc[:,5]

0           경상북도
1             경기
2          광주광역시
3             강원
4             경기
           ...  
1670453    서울특별시
1670454    부산광역시
1670455    광주광역시
1670456      경기도
1670457      경기도
Name: capital_area, Length: 1670283, dtype: object

In [17]:
smp = accounts[['account_id','gender','age']]

In [18]:
smp[['female','male']] = pd.get_dummies(smp.gender)

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
smp.drop('gender',axis=1,inplace=True)

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
smp.head()

,account_id,age,female,male
0,102600,50,1,0
1,103417,40,1,0
2,105247,59,0,1
3,109061,59,1,0
4,109217,46,1,0


In [31]:
products

,product_id,product_name,category_id,published_at,shop_price,maker_name
0,51205644,NCS LH공사(한국토지주택공사) NCS직업기초능력평가,1001,20171025,16000.0,고시넷
1,51205665,NCS K-water 한국수자원공사 NCS직업기초능력평가/직업성격검사,1001,20171025,17000.0,고시넷
2,51205670,심령탐정 야쿠모 12,1008,20170925,4800.0,삼양(만화)
3,51205708,빨리 강해지는 중학 수학 1-1 유형기본서 (2021년용),1003,20170925,14000.0,동아출판
4,51205711,빨리 강해지는 중학 수학 2-1 유형기본서 (2018년),1003,20170928,15000.0,동아출판
...,...,...,...,...,...,...
1725484,85352665,2020 임성래의 쉽게 쓴 품질경영기사 필기,1001,20200106,39000.0,성안당
1725485,85353942,파고다 HSK 5급 종합서 최신 개정판,1011,20200102,22000.0,PAGODA Books
1725486,85354052,2020 고시넷 2주완성 사회조사분석사 2급 필기,1001,20200110,23000.0,고시넷
1725487,85354327,이시원의 영어 대모험 1 : 인칭대명사,1027,20200108,12000.0,아울북


In [34]:
tmp = products[['product_id','shop_price','published_at']]
tmp = tmp.sort_values(by='published_at')

In [35]:
del_1 = list(tmp[:4].index)
del_2 = list(tmp[-879:].index)
del_1.append(del_2)

In [36]:
for i in del_1:
    tmp.drop(i,inplace=True)

In [37]:
tmp.head()

,product_id,shop_price,published_at
1055058,5680,26000.0,19000101.0
1058915,9939,21000.0,19000131.0
335076,3105289,4000.0,19541020.0
1136875,117972,4000.0,19541031.0
1134782,115797,3000.0,19571031.0


###  100개

In [25]:
best100 = list(orders.product_id.value_counts().index[:100])

In [26]:
clicks100 = clicks[clicks.product_id.isin(best100)]
clicks100 = clicks100[['account_id','product_id']]
clicks100

,account_id,product_id
3,101773,84659792
7,101773,86895523
8,101773,81503945
14,101773,81503945
16,102377,81700323
...,...,...
60265352,16938760,85927898
60265358,16942209,89967507
60265398,16955855,86017987
60265399,16955855,90233128


In [27]:
orders100 = orders[orders.product_id.isin(best100)]
orders100 = orders100[['account_id','product_id']]
orders100['purchased'] = 1
orders100

,account_id,product_id,purchased
25,103063,66321042,1
27,103063,84659792,1
36,108787,89309569,1
47,115313,84936431,1
53,115313,80494054,1
...,...,...,...
8382457,16968162,85927898,1
8382486,16976166,65282018,1
8382490,16983122,90233128,1
8382491,16983529,80499154,1


In [28]:
f1 = pd.merge(clicks100,orders100,how='left')
f1 = f1.fillna(0)
f1

,account_id,product_id,purchased
0,101773,84659792,0.0
1,101773,86895523,1.0
2,101773,81503945,0.0
3,101773,81503945,0.0
4,102377,81700323,0.0
...,...,...,...
4051083,16938760,85927898,0.0
4051084,16942209,89967507,1.0
4051085,16955855,86017987,1.0
4051086,16955855,90233128,0.0


In [38]:
f2 = pd.merge(f1,smp,how='left')
f3 = pd.merge(f2,tmp,how='left')
f3.purchased = f3.purchased.astype(int)

In [39]:
f4 = f3.dropna()
f4.female = f4.female.astype(int)
f4.male = f4.male.astype(int)
f4.published_at = pd.to_numeric(f4.published_at).astype(int)
f4.published_at = f4.published_at.astype(str)
f4

,account_id,product_id,purchased,age,female,male,shop_price,published_at
0,101773,84659792,0,47.0,1,0,19800.0,20191224
1,101773,86895523,1,47.0,1,0,13800.0,20200201
2,101773,81503945,0,47.0,1,0,16000.0,20191030
3,101773,81503945,0,47.0,1,0,16000.0,20191030
4,102377,81700323,0,45.0,1,0,12000.0,20191205
...,...,...,...,...,...,...,...,...
4051083,16938760,85927898,0,24.0,1,0,16000.0,20200108
4051084,16942209,89967507,1,47.0,0,1,18000.0,20200420
4051085,16955855,86017987,1,21.0,0,1,19800.0,20200110
4051086,16955855,90233128,0,21.0,0,1,16800.0,20200525


In [40]:
f5 = f4.sort_values(by='published_at')
f5.published_at[-1816342:] = 2020
f5.published_at[:-1816342] = 2019
f5[['2019','2020']] = pd.get_dummies(f5.published_at)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
f5['age'] = pd.DataFrame(mms.fit_transform(f5))[3]
f5['shop_price'] = pd.DataFrame(mms.fit_transform(f5))[6]
f5 = f5.dropna()

In [43]:
f5

,account_id,product_id,purchased,age,female,male,shop_price,published_at,2019,2020
1506733,2524127,1940233,0,0.358696,1,0,0.295154,2019,1,0
3476740,6837411,1940233,1,0.304348,1,0,0.471366,2019,1,0
392103,3675441,1940233,0,0.402174,0,1,0.735683,2019,1,0
392102,3675441,1940233,0,0.195652,0,1,0.735683,2019,1,0
626043,13627830,1940233,0,0.358696,1,0,0.295154,2019,1,0
...,...,...,...,...,...,...,...,...,...,...
1248405,5368542,90248119,0,0.097826,1,0,0.383260,2020,0,1
2626582,11090826,90248119,0,0.489130,1,0,0.127753,2020,0,1
3658890,7610541,90248119,0,0.315217,1,0,0.295154,2020,0,1
1361602,461766,90248119,0,0.304348,1,0,0.735683,2020,0,1


In [47]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(f5.product_id)
f5['product_id'] = encoder.transform(f5.product_id)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [48]:
#f5['F/old'] = f5['female'].mul(f5['2019'])
#f5['F/new'] = f5['female'].mul(f5['2020'])
#f5['M/old'] = f5['male'].mul(f5['2019'])
#f5['M/new'] = f5['male'].mul(f5['2020'])

In [56]:
temp = f5[['product_id','purchased','age','female','shop_price','2020']]

In [57]:
temp

,product_id,purchased,age,female,shop_price,2020
1506733,0,0,0.358696,1,0.295154,0
3476740,0,1,0.304348,1,0.471366,0
392103,0,0,0.402174,0,0.735683,0
392102,0,0,0.195652,0,0.735683,0
626043,0,0,0.358696,1,0.295154,0
...,...,...,...,...,...,...
1248405,98,0,0.097826,1,0.383260,1
2626582,98,0,0.489130,1,0.127753,1
3658890,98,0,0.315217,1,0.295154,1
1361602,98,0,0.304348,1,0.735683,1


In [58]:
temp.purchased.value_counts()/len(dataset)

0    0.744916
1    0.255084
Name: purchased, dtype: float64

In [61]:
pd.get_dummies(temp.product_id)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1506733,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3476740,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
392103,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
392102,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
626043,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248405,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2626582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3658890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1361602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
action = f5[['account_id','product_id']]

In [ ]:
reward = f5['purchased']

In [ ]:
context

In [ ]:
action

In [ ]:
reward

In [245]:
X = context
X['Intercept'] = 0.5
X = X[['Intercept','F/old','F/new','M/old','M/new']]
X = X + 0.00001*np.random.rand(3932825, 5)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [257]:
len(X)

3932825

In [262]:
X

,Intercept,F/old,F/new,M/old,M/new
1506733,0.500008,1.000007,9.561622e-06,7.612026e-06,1.536164e-07
3476740,0.500009,1.000003,9.632183e-07,1.547870e-06,8.176441e-06
392103,0.500003,1.000008,7.047838e-06,1.000006e+00,4.534383e-06
392102,0.500002,1.000007,8.337174e-06,1.000007e+00,1.285882e-06
626043,0.500004,1.000005,2.740969e-06,8.278386e-06,6.251252e-06
...,...,...,...,...,...
1248405,0.500006,1.000008,1.000003e+00,8.525126e-06,5.498712e-06
2626582,0.500006,1.000003,1.000008e+00,7.382458e-06,1.011174e-06
3658890,0.500005,1.000000,1.000000e+00,2.285659e-06,9.123524e-06
1361602,0.500006,1.000001,1.000008e+00,6.064762e-06,3.961457e-06


In [220]:
y = reward

In [221]:
len(X) == len(y)

True

In [231]:
from sklearn.linear_model import LinearRegression

In [264]:
linear_regression = LinearRegression()
linear_regression.fit(X,y)
print('Intercept from library:', linear_regression.intercept_)
print('Slope from library:', linear_regression.coef_[0])

Intercept from library: -18.31809267316646
Slope from library: -63.9281279674045


In [273]:
linear_regression.coef_

array([-6.39281280e+01,  5.04940785e+01,  9.12079590e-02, -5.28377689e-03,
        1.07393995e-01])

## 10개 

In [29]:
best10 = list(orders.product_id.value_counts().index[:10])

In [30]:
clicks10 = clicks[clicks.product_id.isin(best10)]
clicks10 = clicks10[['account_id','product_id']]
clicks10

,account_id,product_id
7,101773,86895523
133,106715,86895523
148,106715,86895523
152,106715,86895523
227,112092,89309569
...,...,...
60265331,16920675,89309569
60265333,16920675,89309569
60265334,16920675,89309569
60265335,16920675,89309569


In [31]:
orders10 = orders[orders.product_id.isin(best10)]
orders10 = orders10[['account_id','product_id']]
orders10['purchased'] = 1
orders10

,account_id,product_id,purchased
36,108787,89309569,1
118,126646,89484458,1
138,130641,89512708,1
243,146186,89484458,1
271,147855,85927912,1
...,...,...,...
8382386,16940895,85927903,1
8382388,16940895,85927912,1
8382389,16940895,85927927,1
8382390,16940895,85927908,1


In [32]:
t1 = pd.merge(clicks10,orders10,how='left')
t1 = t1.fillna(0)
t1

,account_id,product_id,purchased
0,101773,86895523,1.0
1,106715,86895523,1.0
2,106715,86895523,1.0
3,106715,86895523,1.0
4,112092,89309569,1.0
...,...,...,...
668763,16920675,89309569,1.0
668764,16920675,89309569,1.0
668765,16920675,89309569,1.0
668766,16920675,89309569,1.0


In [33]:
t2 = pd.merge(t1,smp,how='left')
t3 = pd.merge(t2,tmp,how='left')
t3.purchased = t3.purchased.astype(int)

In [34]:
t4 = t3.dropna()
t4.female = t4.female.astype(int)
t4.male = t4.male.astype(int)
t4

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,account_id,product_id,purchased,age,female,male,shop_price
0,101773,86895523,1,47.0,1,0,13800.0
4,112092,89309569,1,42.0,0,1,16000.0
5,113364,89484458,0,66.0,0,1,12000.0
6,119143,85927912,0,49.0,1,0,8100.0
7,119143,85927919,0,49.0,1,0,8700.0
...,...,...,...,...,...,...,...
668763,16920675,89309569,1,44.0,1,0,16000.0
668764,16920675,89309569,1,44.0,1,0,16000.0
668765,16920675,89309569,1,44.0,1,0,16000.0
668766,16920675,89309569,1,44.0,1,0,16000.0


In [35]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
t4['age'] = pd.DataFrame(mms.fit_transform(t4))[3]
t4['shop_price'] = pd.DataFrame(mms.fit_transform(t4))[6]
t4 = t4.dropna()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [36]:
t4

,account_id,product_id,purchased,age,female,male,shop_price
0,101773,86895523,1,0.423913,1,0,0.794393
4,112092,89309569,1,0.445652,0,1,0.317757
5,113364,89484458,0,0.445652,0,1,0.224299
6,119143,85927912,0,0.445652,1,0,0.271028
7,119143,85927919,0,0.445652,1,0,0.271028
...,...,...,...,...,...,...,...
647898,12463546,85927919,0,0.391304,1,0,1.000000
647899,12463546,85927912,0,0.391304,1,0,1.000000
647900,12465953,89309569,0,0.391304,0,1,1.000000
647901,12465953,86895523,0,0.391304,0,1,1.000000


In [37]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(t4.product_id)
t4['product_id'] = encoder.transform(t4.product_id)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [38]:
encoder.classes_

array([85927903, 85927905, 85927908, 85927912, 85927919, 85927927,
       86895523, 89309569, 89484458, 89512708], dtype=int64)

In [40]:
t5 = t4[t4.purchased==1]
t6 = t4[t4.purchased==0]

In [41]:
t7 = pd.get_dummies(t5.product_id)

In [42]:
t8=pd.get_dummies(t6.product_id)

In [43]:
t9 = t8.mul(0)

In [44]:
t10 = pd.concat([t7,t9])

In [45]:
t4['order'] = t4.index

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
t10['order'] = t10.index

In [78]:
new_label = t10

In [46]:
X = t4.sort_values(by='order')

In [48]:
y = t10.sort_values(by='order')

In [49]:
X = X[['age','female','male','shop_price']]

In [50]:
y = y[[0,1,2,3,4,5,6,7,8,9]]

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y)

In [53]:
X_train

,age,female,male,shop_price
520379,0.445652,1,0,0.261682
374444,0.434783,1,0,0.224299
332485,0.391304,1,0,0.626168
637963,0.282609,1,0,1.000000
160666,0.173913,0,1,0.271028
...,...,...,...,...
61204,0.369565,1,0,0.794393
638146,0.445652,0,1,0.261682
359239,0.391304,0,1,1.000000
108737,0.326087,1,0,0.626168


In [54]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.0009971312312446684

In [55]:
y_pred = lr.predict(X_test)
r2 = 1 - ((y_test - y_pred)**2).sum() / ((y_test - y_test.mean())**2).sum()
print(r2)

0   -0.000039
1    0.000014
2   -0.000008
3   -0.000020
4   -0.000008
5   -0.000031
6    0.002248
7    0.002349
8    0.001210
9    0.000333
dtype: float64


In [56]:
context = pd.DataFrame(lr.coef_).T

In [57]:
context

,0,1,2,3,4,5,6,7,8,9
0,-8.419015e-04,-5.865679e-05,3.266685e-04,-2.584007e-03,1.667331e-03,-6.471415e-04,2.901510e-03,-8.132976e-04,4.296623e-03,1.038642e-03
1,-3.919286e+08,-2.992464e+08,-2.815237e+08,-5.571137e+08,-3.143082e+08,-2.142522e+08,-9.046195e+08,-1.025715e+09,-7.816549e+08,-7.763197e+08
2,-3.919286e+08,-2.992464e+08,-2.815237e+08,-5.571137e+08,-3.143082e+08,-2.142522e+08,-9.046195e+08,-1.025715e+09,-7.816549e+08,-7.763197e+08
3,7.595420e-04,-5.815029e-04,-7.428527e-04,-9.142160e-04,-1.201034e-04,-7.106662e-04,-1.540303e-03,2.670288e-04,1.476407e-03,1.292467e-03


In [59]:
please = X.T
please.index = [0,1,2,3]

In [60]:
please

,0,4,5,6,7,8,9,10,11,12,...,647893,647894,647895,647896,647897,647898,647899,647900,647901,647902
0,0.423913,0.445652,0.445652,0.445652,0.445652,0.423913,0.423913,0.423913,0.423913,0.423913,...,0.26087,0.26087,0.369565,0.369565,0.391304,0.391304,0.391304,0.391304,0.391304,0.391304
1,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
2,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
3,0.794393,0.317757,0.224299,0.271028,0.271028,0.317757,0.261682,0.261682,0.261682,0.000000,...,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [65]:
context.multiply(please[0],axis='index').values.flatten().shape

(40,)

In [89]:
import re, sys
import os
import numpy

from scipy.sparse import csr_matrix
from scipy.sparse import vstack

In [80]:
c_1 = context.multiply(please[0],axis='index')
n1 = c_1.values.flatten()

In [85]:
n1.shape

(40,)

In [82]:
c_2 = context.multiply(please[4],axis='index')
n2 = c_2.values.flatten()

In [87]:
n3 = np.vstack([n1,n2])

In [88]:
n3.vstack(n2)

AttributeError: 'numpy.ndarray' object has no attribute 'vstack'

In [94]:
temp

<357451x40 sparse matrix of type '<class 'numpy.float64'>'
	with 10571420 stored elements in COOrdinate format>

In [93]:
temp = []
for i in list(please.columns):
    a = context.multiply(please[i],axis='index')
    b = a.values.flatten()
    temp = vstack((temp,b))

KeyboardInterrupt: 

In [512]:
temp

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [513]:
for i in list(please.columns):
    a = context.multiply(please[i],axis='index')
    b = a.values.flatten()
    temp.iloc[i] = b

KeyboardInterrupt: 